In [ ]:
import cv2
import holoocean
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
scenario = "OpenWater-HoveringImagingSonar"
env = holoocean.make(scenario)

env.act('auv0', [0,0,0,0,0,0,0,0])
env.weather.set_fog_density(0.0)

while True:
    state = env.tick()
    if "HorizontalSonar" in state:
        break

In [ ]:
config = holoocean.packagemanager.get_scenario(scenario)
config = config['agents'][0]['sensors'][-1]["configuration"]
horizontal_fov = float(config['Azimuth'])
range_resolution = (config['RangeMax'] - config['RangeMin']) / 512
height = range_resolution * config['RangeBins']
rows = config['RangeBins']
width = np.sin(np.radians(horizontal_fov / 2)) * height * 2
cols = int(np.ceil(width / range_resolution))
bearings = np.radians(np.linspace(-horizontal_fov/2, horizontal_fov/2, config['AzimuthBins']))
f_bearings = interp1d(
            bearings,
            range(len(bearings)),
            kind='linear',
            bounds_error=False,
            fill_value=-1,
            assume_sorted=True)

#build the meshgrid
XX, YY = np.meshgrid(range(cols), range(rows))
x = range_resolution * (rows - YY)
y = range_resolution * (-cols / 2.0 + XX + 0.5)
b = np.arctan2(y, x) * -1
r = np.sqrt(np.square(x) + np.square(y))
map_y = np.asarray(r / range_resolution, dtype=np.float32)
map_x = np.asarray(f_bearings(b), dtype=np.float32)

In [ ]:
img = np.array(state["HorizontalSonar"] * 255).astype(np.uint8)
vis_img = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)

In [ ]:
plt.imshow(vis_img)